In [1]:
import os
%pwd

'f:\\iNeuron\\End2End\\Bappy E-2-E\\End_to_End_ML_Project\\research'

In [2]:
os.chdir('f:\\iNeuron\\End2End\\Bappy E-2-E\\End_to_End_ML_Project')

In [3]:
%pwd

'f:\\iNeuron\\End2End\\Bappy E-2-E\\End_to_End_ML_Project'

In [4]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/Raj-Narayanan-B/End_to_End_ML_Project.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'Raj-Narayanan-B'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'c1b8b6dbed37013669f11f9a86691046a71debcc'

In [5]:
from dataclasses import dataclass #type: ignore
from pathlib import Path #type: ignore

@dataclass(frozen=True)
class ModelEvaluation:
    root_dir: Path
    test_path: Path
    model_name: Path
    metrics: Path
    parameters: dict
    target: str
    mlflow_uri: str


In [6]:
from src.mlproject.constants import *
from src.mlproject.utils.common import load_yaml,create_directories

class ConfigurationManager:
    def __init__(self,
                 config_path = CONFIG_PATH,
                 params_path = PARAMS_PATH,
                 schema_path = SCHEMA_PATH) -> None:
        self.config = load_yaml(config_path)
        self.params = load_yaml(params_path)
        self.schema = load_yaml(schema_path)

    def get_model_evaluation_config(self):
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN
        params = self.params.Elastic_net

        create_directories(config.root_dir)

        model_evaluation_config = ModelEvaluation(
            root_dir = config.root_dir,
            test_path = config.test_path,
            model_name= config.model_name,
            metrics = config.metrics,
            parameters = params,
            target = schema.name,
            mlflow_uri = 'https://dagshub.com/Raj-Narayanan-B/End_to_End_ML_Project.mlflow'
        )

        return model_evaluation_config

In [7]:
from sklearn.metrics import mean_squared_error,r2_score
import numpy as np
import pandas as pd
from pathlib import Path #type: ignore

import mlflow
import mlflow.sklearn
from urllib.parse import urlparse #type: ignore

from src.mlproject.utils.common import load_binary,save_json

class model_evaluation:
    def __init__(self, config: ModelEvaluation) -> None:
        self.config = config

    def eval_metrics(self, y_true, y_pred):
        rmse = np.sqrt(mean_squared_error(y_true=y_true, y_pred=y_pred))
        r2score = r2_score(y_true=y_true, y_pred=y_pred)
        return ({'rmse': rmse,
                 "r2_score": r2score})

    def ml_flow_logging(self):
        test_data = pd.read_csv(self.config.test_path)
        model = load_binary(Path(self.config.model_name))

        xtest = test_data.drop(columns=self.config.target)
        ytest = test_data[self.config.target]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            ypred = model.predict(xtest)

            scores = self.eval_metrics(y_true = ytest, y_pred = ypred)

            save_json(data = scores, path = Path(self.config.metrics))

            mlflow.log_params(self.config.parameters)

            mlflow.log_metric('rmse',scores['rmse'])
            mlflow.log_metric('r2_score',scores['r2_score'])

            if tracking_url_type != 'file':
                mlflow.sklearn.log_model(model,'model',registered_model_name='ElasticNet')
            else:
                mlflow.sklearn.log_model(model,'model')



In [8]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    model_eval_obj = model_evaluation(config=model_eval_config)
    model_eval_obj.ml_flow_logging()

except Exception as e:
    raise e

[2023-11-03 09:40:49,347: INFO: common: The YAML file: config\config.yaml is loaded]
[2023-11-03 09:40:49,350: INFO: common: The YAML file: params.yaml is loaded]
[2023-11-03 09:40:49,353: INFO: common: The YAML file: schema.yaml is loaded]
[2023-11-03 09:40:49,356: INFO: common: Directory:artifacts/model_evaluation is created]
[2023-11-03 09:40:49,506: INFO: common: Object loaded from artifacts\model_trainer\model.joblib]


f:\iNeuron\End2End\Bappy E-2-E\End_to_End_ML_Project\mlproject_1\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator ElasticNet from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[2023-11-03 09:41:02,758: INFO: common: JSON file is saved at]


Successfully registered model 'ElasticNet'.
2023/11/03 09:41:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNet, version 1
Created version '1' of model 'ElasticNet'.
